In [1]:
import logging
import os
import sched
import sys
import time
import pandas as pd
from datetime import datetime
from models.PyCryptoBot import PyCryptoBot
from models.AppState import AppState
from models.Trading import TechnicalAnalysis
from models.TradingAccount import TradingAccount
from models.helper.LogHelper import Logger

app = PyCryptoBot()
account = TradingAccount(app)
technical_analysis = None
state = AppState(app, account)
state.initLastAction()

config = {}
trading_historical_data = {}
dataframe_dict = []

pd.set_option('display.max_columns', None)
# Logger.configure(consoleloglevel="ERROR")

def getAction(now: datetime = datetime.today().strftime('%Y-%m-%d %H:%M:%S'), app: PyCryptoBot = None, price: float = 0,
              df: pd.DataFrame = pd.DataFrame(), df_last: pd.DataFrame = pd.DataFrame(), last_action: str = 'WAIT') -> str:
    ema12gtema26co = bool(df_last['ema12gtema26co'].values[0])
    macdgtsignal = bool(df_last['macdgtsignal'].values[0])
    goldencross = bool(df_last['goldencross'].values[0])
    obv_pc = float(df_last['obv_pc'].values[0])
    elder_ray_buy = bool(df_last['eri_buy'].values[0])
    ema12gtema26 = bool(df_last['ema12gtema26'].values[0])
    macdgtsignalco = bool(df_last['macdgtsignalco'].values[0])
    ema12ltema26co = bool(df_last['ema12ltema26co'].values[0])
    macdltsignal = bool(df_last['macdltsignal'].values[0])
    rri_buy = bool(df_last['rri_buy'].values[0])
    rsi = bool(df_last['rsi14'].values[0])

    action = '' 

    # criteria for a buy signal
    if ema12gtema26co is True \
            and (macdgtsignal is True or app.disableBuyMACD()) \
            and (goldencross is True or app.disableBullOnly()) \
            and (obv_pc > -5 or app.disableBuyOBV()) \
            and (elder_ray_buy is True or app.disableBuyElderRay()) \
            and last_action != 'BUY':

        action = 'BUY'

        Logger.debug('*** Buy Signal ***')
        Logger.debug(f'ema12gtema26co: {ema12gtema26co}')

        if not app.disableBuyMACD():
            Logger.debug(f'macdgtsignal: {macdgtsignal}')

        if not app.disableBullOnly():
            Logger.debug(f'goldencross: {goldencross}')

        if not app.disableBuyOBV():
            Logger.debug(f'obv_pc: {obv_pc} > -5')

        if not app.disableBuyElderRay():
            Logger.debug(f'elder_ray_buy: {elder_ray_buy}')

        Logger.debug(f'last_action: {last_action}')

    elif ema12gtema26 is True \
            and macdgtsignalco is True \
            and (goldencross is True or app.disableBullOnly()) \
            and (obv_pc > -5 or app.disableBuyOBV()) \
            and (elder_ray_buy is True or app.disableBuyElderRay()) \
            and last_action != 'BUY':

        action = 'BUY'

        Logger.debug('*** Buy Signal ***')
        Logger.debug(f'ema12gtema26: {ema12gtema26}')
        Logger.debug(f'macdgtsignalco: {macdgtsignalco}')

        if not app.disableBullOnly():
            Logger.debug(f'goldencross: {goldencross}')

        if not app.disableBuyOBV():
            Logger.debug(f'obv_pc: {obv_pc} > -5')

        if not app.disableBuyElderRay():
            Logger.debug(f'elder_ray_buy: {elder_ray_buy}')

        Logger.debug(f'last_action: {last_action}')

    elif rri_buy is True
            and (goldencross is True or app.disableBullOnly()) \
            and (obv_pc > -5 or app.disableBuyOBV()) \
            and (elder_ray_buy is True or app.disableBuyElderRay()) \
            and last_action != 'BUY':
            and last_action != 'BUY':

        action = 'BUY'

        Logger.debug('*** Buy Signal ***')
        Logger.debug(f'rri_buy: {rri_buy}')

    # criteria for a sell signal
    elif ema12ltema26co is True \
            and (macdltsignal is True or app.disableBuyMACD()) \
            and last_action not in ['', 'SELL']:

        action = 'SELL'

        Logger.debug('*** Sell Signal ***')
        Logger.debug(f'ema12ltema26co: {ema12ltema26co}')
        Logger.debug(f'macdltsignal: {macdltsignal}')
        Logger.debug(f'last_action: {last_action}')

    # anything other than a buy or sell, just wait
    else:
        action = 'WAIT'

    # if disabled, do not buy within 3% of the dataframe close high
    if last_action == 'SELL' and app.disableBuyNearHigh() is True and (price > (df['close'].max() * 0.97)):
        log_text = str(now) + ' | ' + app.getMarket() + ' | ' + \
            app.printGranularity() + ' | Ignoring Buy Signal (price ' + str(price) + ' within 3% of high ' + str(
            df['close'].max()) + ')'
        Logger.warning(log_text)

        action = 'WAIT'

    return action


def getInterval(df: pd.DataFrame = pd.DataFrame(), app: PyCryptoBot = None, iterations: int = 0) -> pd.DataFrame:
    
    return df.iloc[iterations - 1:iterations]
    

def find_high_low_interval(df, start, next_x, price):
    df_clipped = df.iloc[start:start+next_x]
    high_pct = round(((df_clipped["high"].max()/price) - 1) * 100, 2)
    low_pct = round(((df_clipped["low"].min()/price) - 1) * 100, 2)
    return (high_pct, low_pct)

In [2]:
def detect_historical_buys(coin_pair):
    state.iterations = 0
    print("calculating ", coin_pair)
    if coin_pair not in trading_historical_data:
        trading_historical_data[coin_pair] = app.getHistoricalData(coin_pair, 3600)
    trading_data = trading_historical_data[coin_pair]
    trading_dataCopy = trading_data.copy()
    technical_analysis = TechnicalAnalysis(trading_dataCopy)
    technical_analysis.addAll()
    df = technical_analysis.getDataFrame()
    while (state.iterations < 300):
        state.iterations = state.iterations + 1
        df_last = getInterval(df, app, state.iterations)
        if len(df_last) > 0:
            now = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
            price = float(df_last['close'].values[0])
            if price < 0.0001:
                continue
                raise Exception(app.getMarket() + ' is unsuitable for trading, quote price is less than 0.0001!')

            state.action = getAction(now, app, price, df, df_last, state.last_action)
            if state.action=="BUY":
                # technical indicators
                ema12gtema26 = bool(df_last['ema12gtema26'].values[0])
                ema12gtema26co = bool(df_last['ema12gtema26co'].values[0])
                goldencross = bool(df_last['goldencross'].values[0])
                macdgtsignal = bool(df_last['macdgtsignal'].values[0])
                macdgtsignalco = bool(df_last['macdgtsignalco'].values[0])
                ema12ltema26 = bool(df_last['ema12ltema26'].values[0])
                ema12ltema26co = bool(df_last['ema12ltema26co'].values[0])
                macdltsignal = bool(df_last['macdltsignal'].values[0])
                macdltsignalco = bool(df_last['macdltsignalco'].values[0])
                obv = float(df_last['obv'].values[0])
                obv_pc = float(df_last['obv_pc'].values[0])
                elder_ray_buy = bool(df_last['eri_buy'].values[0])
                elder_ray_sell = bool(df_last['eri_sell'].values[0])
                rri = df_last['rri'].values[0]
                last_3_rri_buy = df_last['last_3_rri_buy'].values[0]
                last_5_rri_buy = df_last['last_5_rri_buy'].values[0]
                last_3_rri_sell = df_last['last_3_rri_sell'].values[0]
                last_5_rri_sell = df_last['last_5_rri_sell'].values[0]
                rri_buy = bool(df_last['rri_buy'].values[0])
                rri_sell = df_last['rri_sell'].values[0]
                rsi = df_last['rsi14'].values[0]
                rsi_last = df_last['rsi_last'].values[0]
                rsi_prev = df_last['rsi_prev'].values[0]
                stochrsi14 = float(df_last['stochrsi14'].values[0])
                williamsr14 = float(df_last['williamsr14'].values[0])

                hammer = bool(df_last['hammer'].values[0])
                inverted_hammer = bool(df_last['inverted_hammer'].values[0])
                hanging_man = bool(df_last['hanging_man'].values[0])
                shooting_star = bool(df_last['shooting_star'].values[0])
                three_white_soldiers = bool(df_last['three_white_soldiers'].values[0])
                three_black_crows = bool(df_last['three_black_crows'].values[0])
                morning_star = bool(df_last['morning_star'].values[0])
                evening_star = bool(df_last['evening_star'].values[0])
                three_line_strike = bool(df_last['three_line_strike'].values[0])
                abandoned_baby = bool(df_last['abandoned_baby'].values[0])
                morning_doji_star = bool(df_last['morning_doji_star'].values[0])
                evening_doji_star = bool(df_last['evening_doji_star'].values[0])
                two_black_gapping = bool(df_last['two_black_gapping'].values[0])

                # if simulation interations < 200 set goldencross to true
                if app.isSimulation() and state.iterations < 200:
                    goldencross = True
                buy_score = 0
                if ema12gtema26: buy_score += 3
                if ema12gtema26co: buy_score += 3
                if macdgtsignal: buy_score += 3
                if goldencross: buy_score += 2
                if macdgtsignalco: buy_score +=3
                if obv_pc > -5: buy_score += 3
                if elder_ray_buy: buy_score += 2
                if not elder_ray_sell: buy_score += 2
                if rri_buy and (ema12gtema26 or ema12gtema26co) and (ema12gtema26co or macdgtsignalco):
                    buy_score += 10
                elif rri_buy and (ema12gtema26 and macdgtsignal):
                    buy_score += 8
                elif rri_buy and (ema12gtema26 or macdgtsignal):
                    buy_score += 6
                elif rri_buy:
                    buy_score += 4
                candle_stick_score = 0
                # weak up
                if hammer is True: candle_stick_score += 1
                if inverted_hammer is True: candle_stick_score += 1
                # weak down
                if shooting_star is True: candle_stick_score -= 1
                if hanging_man is True: candle_stick_score -= 1
                # strong up
                if three_white_soldiers is True: candle_stick_score += 2
                if morning_star is True: candle_stick_score += 2
                # strong down
                if three_black_crows is True: candle_stick_score -= 2
                if evening_star is True: candle_stick_score -= 2
                # reliable up
                if three_line_strike is True: candle_stick_score += 3
                if abandoned_baby is True: candle_stick_score += 3
                if morning_doji_star is True: candle_stick_score += 3
                # reliable down
                if evening_doji_star is True: candle_stick_score -= 3
                if two_black_gapping is True: candle_stick_score -= 3
                sarima_3 = None
                sarima_3_margin = None
                sarima_3_dt = None
                
                splitted_df = df.iloc[0:state.iterations]
                prediction_minutes = int(app.getGranularity() / 60) * 3
                if (prediction_minutes<len(splitted_df)):
                    sarima = technical_analysis.customSARIMAPrediction(prediction_minutes,
                                                                             splitted_df)
                    sarima_3 = sarima[1]
                    sarima_3_margin = ((sarima_3 / price) - 1) * 100
                    sarima_3_dt = sarima[0]
                
                
                one_hr_high, one_hr_low = find_high_low_interval(df, state.iterations, 1, price)
                three_hr_high, three_hr_low = find_high_low_interval(df, state.iterations, 3, price)
                six_hr_high, six_hr_low = find_high_low_interval(df, state.iterations, 6, price)
                nine_hr_high, nine_hr_low = find_high_low_interval(df, state.iterations, 9, price)
                twelve_hr_high, twelve_hr_low = find_high_low_interval(df, state.iterations, 12, price)
                
                three_six_high, three_six_low = find_high_low_interval(df, state.iterations+3, 6, price)
                six_12_high, six_12_low = find_high_low_interval(df, state.iterations+6, 12, price)
                dataframe_dict.append({"action": state.action,
                                       "coin_pair": coin_pair,
                                       "timestamp": df_last['date'][0].isoformat(),
                                       "buy_score": buy_score, 
                                       "candle_stick_score":candle_stick_score,
                                       "granularity": app.getGranularity(),
                                       "price": price,
                                       "one_hr_high": one_hr_high, 
                                       "one_hr_low": one_hr_low, 
                                       "three_hr_high": three_hr_high, 
                                       "three_hr_low": three_hr_low, 
                                       "three_six_high": three_six_high,
                                       "three_six_low": three_six_low, 
                                       "six_12_high": six_12_high, 
                                       "six_12_low": six_12_low, 
                                       "six_hr_high": six_hr_high, 
                                       "six_hr_low": six_hr_low,
                                       "nine_hr_high": nine_hr_high, 
                                       "nine_hr_low": nine_hr_low, 
                                       "twelve_hr_high": twelve_hr_high, 
                                       "twelve_hr_low": twelve_hr_low,
                                       "ema12gtema26": ema12gtema26,
                                       "ema12gtema26co": ema12gtema26co,
                                       "goldencross": goldencross,
                                       "macdgtsignal": macdgtsignal, 
                                       "macdgtsignalco": macdgtsignalco, 
                                       "obv": obv, 
                                       "obv_pc": obv_pc, 
                                       "elder_ray_buy": elder_ray_buy,
                                       "elder_ray_sell": elder_ray_sell, 
                                       "rri": rri, 
                                       "rri_buy": rri_buy, 
                                       "rri_sell": rri_sell, 
                                       "last_3_rri_buy": last_3_rri_buy, 
                                       "last_5_rri_buy": last_5_rri_buy, 
                                       "last_3_rri_sell": last_3_rri_sell, 
                                       "last_5_rri_sell": last_5_rri_sell, 
                                       "rsi": rsi,
                                       "rsi_last": rsi_last,
                                       "rsi_prev": rsi_prev,
                                       "stochrsi14": stochrsi14,
                                       "williamsr14": williamsr14,
                                       "sarima_3": sarima_3,
                                       "sarima_3_margin": sarima_3_margin,
                                       "sarima_3_dt": sarima_3_dt,
                                      })
    

In [3]:
%%time
# binance_coin_pairs = ['WTCBNB', 'BATBNB', 'NEOBNB', 'IOTABNB', 'XLMBNB', 'WABIBNB', 'LTCBNB', 'WAVESBNB', 'ICXBNB',
#                           'BLZBNB', 'ZILBNB', 'ONTBNB', 'WANBNB', 'ADABNB', 'ZENBNB', 'EOSBNB', 'THETABNB', 'XRPBNB',
#                           'ENJBNB', 'TRXBNB', 'ETCBNB', 'SCBNB', 'MFTBNB', 'VETBNB', 'RVNBNB', 'MITHBNB', 'BTTBNB',
#                           'HOTBNB', 'FETBNB', 'XMRBNB', 'ZECBNB', 'IOSTBNB', 'CELRBNB', 'DASHBNB', 'MATICBNB',
#                           'ATOMBNB', 'ONEBNB', 'FTMBNB', 'ALGOBNB', 'ANKRBNB', 'WINBNB', 'COSBNB', 'COCOSBNB',
#                           'PERLBNB', 'CHZBNB', 'BANDBNB', 'XTZBNB', 'HBARBNB', 'STXBNB', 'KAVABNB', 'ARPABNB', 'BCHBNB',
#                           'TROYBNB', 'FTTBNB', 'OGNBNB', 'WRXBNB', 'MBLBNB', 'COTIBNB', 'SOLBNB', 'CTSIBNB', 'CHRBNB',
#                           'STMXBNB', 'IQBNB', 'DGBBNB', 'SXPBNB', 'SNXBNB', 'VTHOBNB', 'MKRBNB', 'RUNEBNB', 'FIOBNB',
#                           'AVABNB', 'YFIBNB', 'SRMBNB', 'ANTBNB', 'CRVBNB', 'SANDBNB', 'OCEANBNB', 'NMRBNB',
#                           'DOTBNB', 'LUNABNB', 'RSRBNB', 'PAXGBNB', 'WNXMBNB', 'SUSHIBNB', 'YFIIBNB', 'KSMBNB',
#                           'EGLDBNB', 'DIABNB', 'BELBNB', 'WINGBNB', 'SWRVBNB', 'CREAMBNB', 'UNIBNB', 'AVAXBNB',
#                           'BAKEBNB', 'BURGERBNB', 'CAKEBNB', 'SPARTABNB', 'XVSBNB', 'ALPHABNB', 'AAVEBNB', 'NEARBNB',
#                           'FILBNB', 'INJBNB', 'CTKBNB', 'KP3RBNB', 'AXSBNB', 'HARDBNB', 'UNFIBNB', 'PROMBNB', 'BIFIBNB',
#                           'ICPBNB', 'BTCUSDT', 'ETHUSDT', 'BCCUSDT', 'NEOUSDT', 'LTCUSDT', 'QTUMUSDT', 'ADAUSDT',
#                        'XRPUSDT', 'EOSUSDT', 'IOTAUSDT', 'XLMUSDT', 'ONTUSDT', 'TRXUSDT', 'ETCUSDT',
#                        'ICXUSDT', 'VENUSDT', 'NULSUSDT', 'VETUSDT', 'BCHABCUSDT', 'BCHSVUSDT',
#                        'LINKUSDT', 'WAVESUSDT', 'BTTUSDT', 'USDSUSDT', 'ONGUSDT', 'HOTUSDT', 'ZILUSDT', 'ZRXUSDT',
#                        'FETUSDT', 'BATUSDT', 'XMRUSDT', 'ZECUSDT', 'IOSTUSDT', 'CELRUSDT', 'DASHUSDT', 'NANOUSDT',
#                        'OMGUSDT', 'THETAUSDT', 'ENJUSDT', 'MITHUSDT', 'MATICUSDT', 'ATOMUSDT', 'TFUELUSDT', 'ONEUSDT',
#                        'FTMUSDT', 'ALGOUSDT', 'USDSBUSDT', 'GTOUSDT', 'ERDUSDT', 'DOGEUSDT', 'DUSKUSDT', 'ANKRUSDT',
#                        'WINUSDT', 'COSUSDT', 'NPXSUSDT', 'COCOSUSDT', 'MTLUSDT', 'TOMOUSDT', 'PERLUSDT', 'DENTUSDT',
#                        'MFTUSDT', 'KEYUSDT', 'STORMUSDT', 'DOCKUSDT', 'WANUSDT', 'FUNUSDT', 'CVCUSDT', 'CHZUSDT',
#                        'BANDUSDT', 'BEAMUSDT', 'XTZUSDT', 'RENUSDT', 'RVNUSDT', 'HCUSDT', 'HBARUSDT', 'NKNUSDT',
#                        'STXUSDT', 'KAVAUSDT', 'ARPAUSDT', 'IOTXUSDT', 'RLCUSDT', 'MCOUSDT', 'CTXCUSDT', 'BCHUSDT',
#                        'TROYUSDT', 'VITEUSDT', 'FTTUSDT', 'OGNUSDT', 'DREPUSDT', 'BULLUSDT', 'BEARUSDT',
#                        'ETHBULLUSDT', 'ETHBEARUSDT', 'TCTUSDT', 'WRXUSDT', 'BTSUSDT', 'LSKUSDT', 'BNTUSDT', 'LTOUSDT',
#                        'EOSBULLUSDT', 'EOSBEARUSDT', 'XRPBULLUSDT', 'XRPBEARUSDT', 'STRATUSDT', 'AIONUSDT', 'MBLUSDT',
#                        'COTIUSDT', 'STPTUSDT', 'WTCUSDT', 'DATAUSDT', 'XZCUSDT',
#                        'SOLUSDT', 'CTSIUSDT', 'HIVEUSDT', 'CHRUSDT', 'GXSUSDT', 'ARDRUSDT',
#                        'LENDUSDT', 'MDTUSDT', 'STMXUSDT', 'KNCUSDT', 'REPUSDT', 'LRCUSDT', 'PNTUSDT', 'COMPUSDT',
#                        'BKRWUSDT', 'SCUSDT', 'ZENUSDT', 'SNXUSDT', 'VTHOUSDT', 'DGBUSDT', 'GBPUSDT', 'SXPUSDT',
#                        'MKRUSDT', 'DCRUSDT', 'STORJUSDT', 'MANAUSDT', 'YFIUSDT', 'BALUSDT', 'BLZUSDT', 'IRISUSDT', 'KMDUSDT',
#                        'SRMUSDT', 'ANTUSDT', 'CRVUSDT', 'SANDUSDT', 'OCEANUSDT', 'NMRUSDT', 'DOTUSDT',
#                        'LUNAUSDT', 'RSRUSDT', 'WNXMUSDT', 'TRBUSDT', 'BZRXUSDT', 'SUSHIUSDT', 'YFIIUSDT',
#                        'KSMUSDT', 'EGLDUSDT', 'DIAUSDT', 'RUNEUSDT', 'FIOUSDT', 'UMAUSDT', 'BELUSDT',
#                        'WINGUSDT', 'UNIUSDT', 'NBSUSDT', 'OXTUSDT', 'SUNUSDT', 'AVAXUSDT',
#                        'HNTUSDT', 'FLMUSDT', 'ORNUSDT', 'UTKUSDT', 'XVSUSDT', 'ALPHAUSDT',
#                        'AAVEUSDT', 'NEARUSDT', 'FILUSDT',
#                         'INJUSDT', 'AUDIOUSDT', 'CTKUSDT',
#                        'AKROUSDT', 'AXSUSDT', 'HARDUSDT', 'DNTUSDT', 'STRAXUSDT', 'UNFIUSDT', 'ROSEUSDT', 'AVAUSDT',
#                        'XEMUSDT', 'SKLUSDT', 'SUSDUSDT',
#                         'GRTUSDT', 'JUVUSDT', 'PSGUSDT', '1INCHUSDT', 'REEFUSDT', 'OGUSDT',
#                        'ATMUSDT', 'ASRUSDT', 'CELOUSDT', 'RIFUSDT', 'BTCSTUSDT', 'TRUUSDT', 'CKBUSDT', 'TWTUSDT',
#                        'FIROUSDT', 'LITUSDT', 'SFPUSDT', 'DODOUSDT', 'CAKEUSDT', 'ACMUSDT', 'BADGERUSDT', 'FISUSDT',
#                        'OMUSDT', 'PONDUSDT', 'DEGOUSDT', 'ALICEUSDT', 'LINAUSDT', 'PERPUSDT', 'RAMPUSDT', 'SUPERUSDT',
#                        'CFXUSDT', 'EPSUSDT', 'AUTOUSDT', 'TKOUSDT', 'PUNDIXUSDT', 'TLMUSDT', 'BTGUSDT', 'MIRUSDT', 'BARUSDT', 'FORTHUSDT', 'BAKEUSDT', 'BURGERUSDT',
#                        'SLPUSDT', 'SHIBUSDT', 'ICPUSDT', 'ARUSDT', 'POLSUSDT', 'MDXUSDT', 'MASKUSDT', 'BTCBUSD', 'XRPBUSD', 'ETHBUSD', 'LTCBUSD', 'LINKBUSD', 'ETCBUSD', 'TRXBUSD',
#                        'EOSBUSD', 'XLMBUSD', 'ADABUSD', 'BCHBUSD', 'QTUMBUSD', 'VETBUSD', 'ICXBUSD',
#                        'BNTBUSD', 'ATOMBUSD', 'DASHBUSD', 'NEOBUSD', 'WAVESBUSD', 'XTZBUSD', 'BATBUSD', 'ENJBUSD',
#                        'NANOBUSD', 'ONTBUSD', 'RVNBUSD', 'ALGOBUSD', 'BTTBUSD', 'TOMOBUSD', 'XMRBUSD', 'ZECBUSD',
#                        'DATABUSD', 'SOLBUSD', 'CTSIBUSD', 'HBARBUSD', 'MATICBUSD', 'WRXBUSD', 'ZILBUSD', 'KNCBUSD',
#                        'LRCBUSD', 'IQBUSD', 'GBPBUSD', 'DGBBUSD', 'COMPBUSD', 'SXPBUSD', 'SNXBUSD', 'MKRBUSD',
#                        'RUNEBUSD', 'MANABUSD', 'DOGEBUSD', 'ZRXBUSD', 'FIOBUSD', 'AVABUSD', 'IOTABUSD',
#                        'BALBUSD', 'YFIBUSD', 'SRMBUSD', 'ANTBUSD', 'CRVBUSD', 'SANDBUSD', 'OCEANBUSD',
#                        'NMRBUSD', 'DOTBUSD', 'LUNABUSD', 'IDEXBUSD', 'RSRBUSD', 'TRBBUSD', 'BZRXBUSD', 'SUSHIBUSD',
#                        'YFIIBUSD', 'KSMBUSD', 'EGLDBUSD', 'DIABUSD', 'BELBUSD', 'SWRVBUSD', 'WINGBUSD', 'CREAMBUSD',
#                        'UNIBUSD', 'AVAXBUSD', 'FLMBUSD', 'CAKEBUSD', 'XVSBUSD', 'ALPHABUSD', 'VIDTBUSD', 'AAVEBUSD',
#                        'NEARBUSD', 'FILBUSD', 'INJBUSD', 'AERGOBUSD', 'ONEBUSD', 'AUDIOBUSD', 'CTKBUSD', 'KP3RBUSD',
#                        'AXSBUSD', 'HARDBUSD', 'DNTBUSD', 'CVPBUSD', 'STRAXBUSD', 'FORBUSD', 'UNFIBUSD', 'FRONTBUSD',
#                        'BCHABUSD', 'ROSEBUSD', 'SYSBUSD', 'HEGICBUSD', 'PROMBUSD', 'SKLBUSD', 'COVERBUSD', 'GHSTBUSD',
#                        'DFBUSD', 'JUVBUSD', 'PSGBUSD', 'BTCSTBUSD', 'TRUBUSD', 'DEXEBUSD',
#                        'CKBBUSD', 'TWTBUSD', 'LITBUSD', 'SFPBUSD', 'FXSBUSD', 'DODOBUSD', 'BAKEBUSD',
#                        'UFTBUSD', '1INCHBUSD', 'BANDBUSD', 'GRTBUSD', 'IOSTBUSD', 'OMGBUSD', 'REEFBUSD', 'ACMBUSD',
#                        'AUCTIONBUSD', 'PHABUSD', 'TVKBUSD', 'BADGERBUSD', 'FISBUSD', 'OMBUSD', 'PONDBUSD', 'DEGOBUSD',
#                        'ALICEBUSD', 'CHZBUSD', 'BIFIBUSD', 'LINABUSD', 'PERPBUSD', 'RAMPBUSD', 'SUPERBUSD', 'CFXBUSD',
#                        'XVGBUSD', 'EPSBUSD', 'AUTOBUSD', 'TKOBUSD', 'TLMBUSD', 'BTGBUSD', 'HOTBUSD', 'MIRBUSD',
#                        'BARBUSD', 'FORTHBUSD', 'BURGERBUSD', 'SLPBUSD', 'SHIBBUSD', 'ICPBUSD', 'ARBUSD', 'POLSBUSD',
#                        'MDXBUSD', 'MASKBUSD']
binance_coin_pairs = ['WTCBNB', 'BATBNB', 'NEOBNB', 'IOTABNB']
dataframe_dict = []
for coin_pair in binance_coin_pairs:
    try:
        detect_historical_buys(coin_pair)
    except Exception as e:
        print(e)
buy_dataframe = pd.DataFrame(dataframe_dict)


calculating  WTCBNB


/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to 

/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to 

/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to 

calculating  BATBNB


...

/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to 

/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to 

/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to 

CPU times: user 21min 5s, sys: 6min 54s, total: 28min
Wall time: 5min 25s


/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


In [4]:
buy_dataframe

,action,coin_pair,timestamp,buy_score,candle_stick_score,granularity,price,one_hr_high,one_hr_low,three_hr_high,three_hr_low,three_six_high,three_six_low,six_12_high,six_12_low,six_hr_high,six_hr_low,nine_hr_high,nine_hr_low,twelve_hr_high,twelve_hr_low,ema12gtema26,ema12gtema26co,goldencross,macdgtsignal,macdgtsignalco,obv,obv_pc,elder_ray_buy,elder_ray_sell,rri,rri_buy,rri_sell,last_3_rri_buy,last_5_rri_buy,last_3_rri_sell,last_5_rri_sell,rsi,rsi_last,rsi_prev,stochrsi14,williamsr14,sarima_3,sarima_3_margin,sarima_3_dt
0,BUY,WTCBNB,2021-05-31T20:00:00,5,0,3600,0.002432,0.66,0.49,0.66,-1.52,0.58,-1.60,1.73,-3.78,0.66,-1.60,0.66,-1.60,1.73,-1.64,False,False,False,False,False,20.6,0.00,False,False,NaN,False,False,False,False,False,False,50.000000,NaN,NaN,0.500000,-50.000000,NaN,NaN,None
1,BUY,WTCBNB,2021-05-31T21:00:00,19,0,3600,0.002444,0.08,-2.00,0.08,-2.00,0.08,-2.09,1.23,-4.26,0.08,-2.09,0.08,-2.09,1.23,-2.99,True,True,False,True,True,129.5,528.64,True,False,-1.000000,False,False,False,False,False,False,50.000000,50.000000,NaN,0.500000,-50.000000,NaN,NaN,None
2,BUY,WTCBNB,2021-05-31T22:00:00,9,0,3600,0.002446,0.00,0.00,0.00,-2.17,1.14,-2.04,1.14,-4.33,0.00,-2.17,1.14,-2.17,1.14,-3.07,True,False,False,True,False,421.3,225.33,False,True,0.686275,False,False,False,False,False,False,50.000000,50.000000,50.000000,0.500000,-50.000000,NaN,NaN,None
3,BUY,WTCBNB,2021-05-31T23:00:00,9,0,3600,0.002446,-0.70,-1.92,0.00,-2.17,1.14,-2.21,1.14,-4.33,0.00,-2.17,1.14,-2.21,1.14,-3.56,True,False,False,True,False,421.3,0.00,False,True,NaN,False,False,False,False,False,False,50.000000,50.000000,50.000000,0.500000,-50.000000,NaN,NaN,None
4,BUY,WTCBNB,2021-06-01T00:00:00,6,0,3600,0.002429,-0.21,-1.48,0.70,-1.48,1.85,-2.39,1.85,-3.66,0.70,-1.48,1.85,-2.39,1.85,-3.01,True,False,False,True,False,-1609.5,-482.03,False,True,1.000000,False,False,False,False,False,False,50.000000,50.000000,50.000000,0.500000,-50.000000,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,BUY,IOTABNB,2021-06-13T03:00:00,6,0,3600,0.002999,2.17,0.00,2.17,-0.47,2.30,0.57,NaN,NaN,2.30,-0.47,2.30,-0.47,2.30,-0.47,False,False,False,True,False,-640613.6,0.52,False,True,-0.080000,False,False,True,True,False,False,46.487564,47.725312,54.843987,0.446697,0.636364,0.002993,-0.205438,2021-06-13 06:00:00
1196,BUY,IOTABNB,2021-06-13T04:00:00,10,0,3600,0.003050,-0.07,-2.13,0.59,-2.13,NaN,NaN,NaN,NaN,0.59,-2.13,0.59,-2.13,0.59,-2.13,False,False,False,True,False,-634724.7,-0.92,True,False,0.769231,False,True,True,True,True,True,52.165592,46.487564,47.725312,0.810837,0.363636,0.002962,-2.879845,2021-06-13 07:00:00
1197,BUY,IOTABNB,2021-06-13T05:00:00,6,0,3600,0.003004,0.97,0.07,2.13,0.07,NaN,NaN,NaN,NaN,2.13,0.07,2.13,0.07,2.13,0.07,False,False,False,True,False,-643414.0,1.37,False,True,-0.698413,False,False,False,True,True,True,47.291454,52.165592,46.487564,0.498251,0.609626,0.002973,-1.029224,2021-06-13 08:00:00
1198,BUY,IOTABNB,2021-06-13T06:00:00,8,0,3600,0.003032,1.19,-0.53,1.19,-0.53,NaN,NaN,NaN,NaN,1.19,-0.53,1.19,-0.53,1.19,-0.53,False,False,False,True,False,-639775.4,-0.57,True,True,0.962963,False,True,False,True,True,True,50.333476,47.291454,52.165592,0.705614,0.445055,0.002968,-2.109405,2021-06-13 09:00:00


In [5]:
buy_dataframe.describe()

,buy_score,candle_stick_score,granularity,price,one_hr_high,one_hr_low,three_hr_high,three_hr_low,three_six_high,three_six_low,six_12_high,six_12_low,six_hr_high,six_hr_low,nine_hr_high,nine_hr_low,twelve_hr_high,twelve_hr_low,obv,obv_pc,rri,rsi,rsi_last,rsi_prev,stochrsi14,williamsr14,sarima_3,sarima_3_margin
count,1200.000000,1200.000000,1200.0,1200.000000,1196.000000,1196.000000,1196.000000,1196.000000,1184.000000,1184.000000,1172.000000,1172.000000,1196.000000,1196.000000,1196.000000,1196.000000,1196.000000,1196.000000,1200.000000,1.200000e+03,1151.000000,1200.000000,1196.000000,1192.000000,1200.000000,1200.000000,480.000000,480.000000
mean,7.907500,-0.019167,3600.0,0.037529,0.687868,-0.702793,1.315134,-1.286070,1.729054,-1.934079,2.246980,-2.793532,1.849699,-1.805610,2.217801,-2.196898,2.499649,-2.530970,-64018.653917,-1.364632e+12,-0.045382,46.217375,46.213885,46.215564,0.449597,-1.573525,0.036372,-0.071994
std,3.633173,0.349621,0.0,0.060918,1.082693,0.752452,1.624162,1.039626,2.715805,1.834925,3.689271,2.475083,2.099277,1.304731,2.475782,1.534712,2.797779,1.718654,177888.328197,4.727224e+13,0.618624,8.929766,8.940590,8.951692,0.326226,10.313952,0.058978,1.508336
min,0.000000,-2.000000,3600.0,0.001785,-1.770000,-4.680000,-1.530000,-7.870000,-3.810000,-8.800000,-4.330000,-10.750000,-1.300000,-7.960000,-1.300000,-8.800000,-1.300000,-9.340000,-726429.900000,-1.637558e+15,-1.000000,15.603670,15.603670,15.603670,0.000000,-50.000000,0.001770,-7.448212
25%,6.000000,0.000000,3600.0,0.001997,0.090000,-1.060000,0.360000,-1.830000,0.080000,-2.950000,0.050000,-4.100000,0.550000,-2.450000,0.690000,-2.980000,0.840000,-3.340000,-64073.000000,-3.860000e+00,-0.577352,40.750249,40.738298,40.738298,0.158920,0.396395,0.001946,-0.875253
50%,7.000000,0.000000,3600.0,0.002703,0.460000,-0.570000,0.895000,-1.110000,1.230000,-1.830000,1.420000,-2.700000,1.325000,-1.600000,1.520000,-1.940000,1.690000,-2.250000,-13145.405000,2.000000e-02,-0.100000,46.431434,46.411315,46.382951,0.460983,0.635244,0.002701,-0.161014
75%,10.000000,0.000000,3600.0,0.035697,0.990000,-0.220000,1.765000,-0.550000,2.590000,-0.830000,3.445000,-1.300000,2.390000,-0.890000,2.882500,-1.100000,3.250000,-1.290000,-4530.355000,4.982500e+00,0.500000,51.134048,51.134048,51.192669,0.705736,0.820056,0.035513,0.632536
max,21.000000,5.000000,3600.0,0.158200,13.950000,2.140000,15.170000,1.740000,20.980000,6.380000,22.900000,7.800000,19.690000,1.740000,20.980000,1.740000,22.000000,1.740000,289495.700000,1.109550e+03,1.000000,78.995698,78.995698,78.995698,1.000000,1.000000,0.149586,5.103172


In [26]:
buy_dataframe.query("rsi<30").describe()

,buy_score,candle_stick_score,granularity,price,one_hr_high,one_hr_low,three_hr_high,three_hr_low,three_six_high,three_six_low,six_12_high,six_12_low,six_hr_high,six_hr_low,nine_hr_high,nine_hr_low,twelve_hr_high,twelve_hr_low,obv,obv_pc,rri,rsi,rsi_last,rsi_prev,stochrsi14,williamsr14,sarima_3,sarima_3_margin
count,41.000000,41.000000,41.0,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,40.000000,40.00000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,39.000000,41.000000,41.000000,41.000000,41.000000,41.000000,1.000000,1.000000
mean,5.268293,-0.097561,3600.0,0.043429,1.040000,-0.824146,2.095122,-1.700976,2.733171,-2.401220,4.789250,-2.83000,2.552195,-2.336341,3.335366,-3.025854,4.118293,-3.502683,-57534.720244,7.664146,-0.459630,24.811641,26.653931,28.449924,0.080242,0.943375,0.133101,0.453951
std,2.578996,0.436170,0.0,0.064856,1.412393,0.867136,1.717794,1.415118,2.656610,2.389085,3.677619,3.40083,1.877205,1.934186,2.252453,1.872610,3.087600,2.180858,114334.122834,19.294218,0.379845,4.074693,5.539061,7.011432,0.117438,0.062176,NaN,NaN
min,0.000000,-2.000000,3600.0,0.001968,-0.350000,-2.770000,0.000000,-4.870000,-1.760000,-7.000000,-0.790000,-8.41000,0.090000,-7.000000,0.090000,-7.000000,0.180000,-7.030000,-465215.400000,-30.560000,-1.000000,15.603670,15.603670,15.603670,0.000000,0.741294,0.133101,0.453951
25%,3.000000,0.000000,3600.0,0.002131,0.200000,-1.440000,0.710000,-2.740000,1.310000,-3.820000,1.507500,-5.89000,1.060000,-3.650000,1.610000,-4.240000,1.660000,-5.230000,-42634.000000,0.980000,-0.764286,21.395516,22.283818,22.705850,0.000000,0.900000,0.133101,0.453951
50%,5.000000,0.000000,3600.0,0.002248,0.620000,-0.660000,1.740000,-1.510000,2.500000,-2.650000,5.265000,-2.45500,1.910000,-1.530000,3.190000,-3.020000,3.530000,-3.510000,-29912.400000,3.460000,-0.482759,25.999051,27.863451,29.655512,0.000000,0.953488,0.133101,0.453951
75%,7.000000,0.000000,3600.0,0.139400,1.150000,-0.180000,2.880000,-0.680000,4.760000,-1.050000,7.182500,-0.03500,4.240000,-0.750000,4.880000,-1.530000,5.850000,-1.730000,-7109.140000,9.590000,-0.150758,28.238606,30.480014,33.033085,0.177943,1.000000,0.133101,0.453951
max,11.000000,0.000000,3600.0,0.149100,5.820000,0.890000,5.820000,0.890000,10.590000,2.040000,11.760000,2.80000,5.820000,0.000000,10.590000,0.000000,11.760000,0.000000,-1842.200000,99.760000,0.205479,29.992024,36.726067,50.000000,0.330898,1.000000,0.133101,0.453951


In [6]:
# buy_dataframe.to_csv("buy_df_1hr_ema_rri_rsi_rules.csv")

### Observation

**In the first hour,** 

- 50% of the BUY signals give 1.11% profit. Setting sellupprpcnt with 1% will get 50% success rate in first hour
- 75% of the buy signals give 0.45% profit. Setting sellupprpcnt with 0.4% will get 75% success rate in first hour


- If invested Rs.100 and for 100 buy signals sellupprpcnt is 0.45, definite chance of making Rs.139 if 75% is succesful
- If invested Rs.100 and for 100 buy signals sellupprpcnt is 1.11, definite chance of making Rs.171 if 50% is succesful

- Filterd by ones that makes < 0.05 in first hour and six_hr_high is also less than 0.00 margin, there is none. Which means, even if it does. not make more than 0.05 margin in first hour, at some point it makes more than 0% profit

**In the three hours,**

- 50% of the BUY signals give 2.0% profit. Setting sellupprpcnt with 1% will get 50% success rate in first hour
- 75% of the buy signals give 0.8% profit. Setting sellupprpcnt with 0.4% will get 75% success rate in first hour

